In [ ]:
%load_ext autoreload
%autoreload 2

from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import seaborn as sns
from models.transformer import Transformer
from train import Trainer
from dataloader import EnglishToSpanish

# Model

In [ ]:
data = EnglishToSpanish()

In [ ]:
tm = Transformer(data.voc_src_len, data.voc_tgt_len)

In [ ]:
trainer = Trainer()
_, _ = trainer.fit(tm, data)

# Inference

## Custom sentence

In [ ]:
sentence = "ey! hola"
sentence = data._tokenize_es(sentence)
inp_pad = data.MAX_LEN - len(sentence)
sentence = sentence + ["<PAD>"] * inp_pad
inp = data._token_to_idx(sentence).reshape(1, -1)

## Sentence from training

In [ ]:
dl = iter(DataLoader(data))

In [ ]:
inp, out, inp_pad, out_pad, src, tgt = next(dl)
print(data._idx_to_token(inp, is_src=True))
print(data._idx_to_token(out))

## Inference

In [ ]:
tm.eval()
sos = torch.repeat_interleave(data.sos, 1, 0)
out1 = torch.cat([sos], 1)
print(out1)
print(data._idx_to_token(out1))

In [ ]:
PICK_TOP = 0

R = tm(inp, out1, inp_pad, 0)
last_pred = R[0,-1]
values, indices = torch.topk(last_pred, k=2)
r = torch.tensor([indices[PICK_TOP]])
out1 = torch.cat([out1, r.reshape(1, 1)], 1)
print(f"Adding with prob. {values[PICK_TOP]}")
print(out1)
print(data._idx_to_token(out1))

In [ ]:
print(data.voc_tgt)
print(last_pred)